In [1]:
# We will do the imports required for this notebook here
%load_ext autoreload
%autoreload 2
# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# swift is a lightweight browser-based simulator which comes eith the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# import urchin to read urdf files
from urchin import URDF

# this package provides several solvers for solving quadratic programmes
import qpsolvers as qp 

# modern robotics code library accompanying 
# Modern Robotics: Mechanics, Planning, and Control (Kevin Lynch and Frank Park)
import modern_robotics as mr

# the os package provides utilities for interacting with the operating system
import os

# import config 
from config.config import * 
from models.panda_model import *
from planning.screw import *
import pytransform3d as pt3d

In [2]:

hl, hw, hh = 0.1/2, 0.07/2, 0.03/2
# 8 corners of the box
corners = [
    (-hl, hw, -hh),     # 0: back-left-bottom
    (-hl, -hw, -hh),    # 1: back-right-bottom
    (hl, -hw, -hh),     # 2: front-right-bottom
    (hl, hw, -hh),     # 3: front-left-bottom
    (-hl, hw, hh),      # 4: back-left-top
    (-hl, -hw, hh),     # 5: back-right-top
    (hl, -hw, hh),      # 6: front-right-top
    (hl, hw, hh)        # 7: front-left-top
]


# 12 edges defined by pairs of corner indices
edge_indices = [
    (0, 1), (1, 2), (2, 3), (3, 0),  # bottom face
    (4, 5), (5, 6), (6, 7), (7, 4),  # top face
    (0, 4), (1, 5), (2, 6), (3, 7)   # connecting edges
]

edges = [(np.array(corners[i]), np.array(corners[j])) for i, j in edge_indices]

start, end = edges[3]
start, end
wTb = sm.SE3(0.7, 0, 0.015)
wTb * end, start

# create object to grasp
box = sg.Cuboid(scale=[0.1, 0.07, 0.03],
                color='blue')
box.T = sm.SE3(0.7, 0, 0.015)
box.set_alpha(0.5)

pose = SE3(box.T) * sm.SE3.Rx(np.pi/2)

SE3(sm.SO3(pose.R))  * SE3(start), SE3(start) * SE3(sm.SO3(pose.R)), start

(   1         0         0         0.05      
   0         0        -1         0.015     
   0         1         0         0.035     
   0         0         0         1         
,
    1         0         0         0.05      
   0         0        -1         0.035     
   0         1         0        -0.015     
   0         0         0         1         
,
 array([ 0.05 ,  0.035, -0.015]))

In [51]:
SE3(start)

   1         0         0         0.05      
   0         1         0         0.035     
   0         0         1        -0.015     
   0         0         0         1         


In [47]:
radius = 0.037
height = 0.234

cylinder = sg.Cylinder(radius=radius, length=height, 
                       pose=SE3(0.1, 0, height/2) * SE3.Rx(np.pi/6), color='purple')
cylinder.set_alpha(0.5)
env = Swift()
wTc = sg.Axes(length=0.1)
wTc.T = cylinder.T
env.launch(realtime=True, browser="notebook")
env.add(cylinder)
env.add(wTc)
env.add(box)

2

In [50]:
sm.SO3(SE3(cylinder.T).R) * np.array([0, 0, 1])

array([[ 0.       ],
       [-0.5      ],
       [ 0.8660254]])